In [ ]:
# !pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121

In [1]:
# !pip install -q datasets transformers accelerate loralib bitsandbytes sentencepiece gradio fire peft wandb

In [131]:
import zipfile
import json
import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import evaluate
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from loguru import logger
import numpy as np

In [6]:
print(torch.cuda.is_available())

True


In [7]:
data_zip = zipfile.ZipFile('garpix-solution\data\garpix_data.zip', 'r')

In [8]:
with data_zip.open(data_zip.filelist[0].filename) as file:
    json_file = json.loads(file.read())

In [9]:
boxes = pd.json_normalize(json_file['first_visual']['boxes'])[[
    'mass','size.width','size.height', 'size.length'
    ]].to_numpy().flatten()

In [10]:
target = json_file['first_visual']['calculation_info']['density_percent']

In [11]:
def json_to_array(json):
    
    boxes = pd.json_normalize(json['first_visual']['boxes'])[[
        'mass', 'size.width', 'size.height', 'size.length'
        ]]
    target = json['first_visual']['calculation_info']['density_percent']
    
    return boxes.to_numpy().flatten(), target

In [94]:
arr = []
targets = []

for data_info in data_zip.filelist:
    with data_zip.open(data_info.filename) as file:
        json_file = json.loads(file.read())
        try:
            boxes, target = json_to_array(json_file)
            # if len(boxes) <= 512:
            boxes = np.array2string(
                boxes, separator=' ', formatter={'float_kind': lambda x: str(int(x))}
            ).replace('[', '').replace(']', '').replace('\n', '')
            arr.append(str(boxes))
            targets.append(round(target))
            # else:
            #     logger.info(f'{data_info.filename} length is more than 512. Skip.')
            #     pass
        except Exception as e:
            logger.warning(f'\nError {e} \n In file {data_info.filename}')
            pass

In [95]:
len(arr), len(targets)

(2565, 2565)

In [96]:
df = pd.DataFrame()

In [97]:
df['sequence'] = arr
df['labels'] = targets

In [121]:
len(set(targets))

52

In [126]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
train_data = Dataset.from_pandas(train)
test_data = Dataset.from_pandas(test)

In [127]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

def tokenize_function(examples):
    return tokenizer(examples['sequence'], padding="max_length", truncation=True)

In [129]:
train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2052 [00:00<?, ? examples/s]

Map:   0%|          | 0/513 [00:00<?, ? examples/s]

In [132]:
model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", 
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(set(targets)))

training_args = TrainingArguments(output_dir="test_trainer",
                                  no_cuda=False,
                                  push_to_hub=False,
                                  num_train_epochs=1,
                                  evaluation_strategy='epoch')

metric = evaluate.load("accuracy")

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [133]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [134]:
"cuda:0" if torch.cuda.is_available() else "cpu"

'cuda:0'

In [135]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [136]:
trainer.train()

  0%|          | 0/257 [00:00<?, ?it/s]